In [4]:
%pip install pandas
%pip install scikit-learn
%pip install matplotlib
%pip install seaborn
%pip install xgboost --no-binary xgboost -v

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Applications/Xcode.app/Contents/Developer/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Applications/Xcode.app/Contents/Developer/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Applications/Xcode.app/Contents/Developer/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Applications/Xcode.app/Contents/Develop

In [5]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_log_error, mean_absolute_error


import seaborn as sns
import matplotlib.pyplot as plt

import xgboost as xgb

XGBoostError: 
XGBoost Library (libxgboost.dylib) could not be loaded.
Likely causes:
  * OpenMP runtime is not installed
    - vcomp140.dll or libgomp-1.dll for Windows
    - libomp.dylib for Mac OSX
    - libgomp.so for Linux and other UNIX-like OSes
    Mac OSX users: Run `brew install libomp` to install OpenMP runtime.

  * You are running 32-bit Python on a 64-bit OS

Error message(s): ["dlopen(/Users/fabri/Library/Python/3.9/lib/python/site-packages/xgboost/lib/libxgboost.dylib, 0x0006): Library not loaded: @rpath/libomp.dylib\n  Referenced from: <89AD948E-E564-3266-867D-7AF89D6488F0> /Users/fabri/Library/Python/3.9/lib/python/site-packages/xgboost/lib/libxgboost.dylib\n  Reason: tried: '/opt/homebrew/opt/libomp/lib/libomp.dylib' (no such file), '/System/Volumes/Preboot/Cryptexes/OS/opt/homebrew/opt/libomp/lib/libomp.dylib' (no such file), '/opt/homebrew/opt/libomp/lib/libomp.dylib' (no such file), '/System/Volumes/Preboot/Cryptexes/OS/opt/homebrew/opt/libomp/lib/libomp.dylib' (n

In [4]:
data = pd.read_csv('./Train.csv')
data.head()

/var/folders/t2/yz2y5gb1397dsm77lnp4kz0m0000gn/T/ipykernel_81493/2216384314.py:1: DtypeWarning: Columns (1,3,4,5,6,7,8,9,10,11,12,13,14,15) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('./Train.csv')


,id,realSum,room_type,room_shared,room_private,person_capacity,host_is_superhost,multi,biz,cleanliness_rating,guest_satisfaction_overall,bedrooms,dist,metro_dist,lng,lat,city,day_type
0,1,210.3658537,Entire home/apt,FALSE,FALSE,2,FALSE,0,1,9,91,0,0.541021778,0.42791926,-9.13395,38.71038,lisbon,weekday
1,2,176.1817209,Private room,FALSE,TRUE,2,FALSE,1,0,9,89,1,3.032839151,0.343492056,2.14165,41.37011,barcelona,weekday
2,3,142.0541405,Private room,FALSE,TRUE,2,TRUE,0,0,10,98,1,11.90971188,6.710908979,0.00475,51.44025,london,weekend
3,4,428.7445242,Entire home/apt,FALSE,FALSE,4,FALSE,0,0,9,94,0,4.158389765,0.145570338,2.30179,48.87297,paris,weekend
4,5,220.2798022,Entire home/apt,FALSE,FALSE,6,TRUE,0,1,10,93,2,1.302189968,0.369474779,23.72468,37.96746,athens,weekend


In [5]:
# feature types
numerical_features = [
    'person_capacity', 'cleanliness_rating', 'guest_satisfaction_overall',
    'bedrooms', 'dist', 'metro_dist', 'lng', 'lat'
]

categorical_features = [
    'room_type', 'day_type', 'city'
]

binary_features = [
    'room_shared', 'room_private', 'host_is_superhost', 'multi', 'biz'
]

In [9]:
# remove bad row
data = data[data['id'] != 2376]

# Normalize the price
data['realSum'] = data['realSum'].astype(float).round().astype(int)
data['log_price'] = np.log1p(data['realSum'])

# Convert 'FALSE' and 'TRUE' strings to 0 and 1, and booleans False and True to 0 and 1 for "room_shared" and "room_private"
data['room_shared'] = data['room_shared'].replace({'FALSE': 0, 'TRUE': 1, False: 0, True: 1})
data['room_private'] = data['room_private'].replace({'FALSE': 0, 'TRUE': 1, False: 0, True: 1})
data['host_is_superhost'] = data['room_private'].replace({'FALSE': 0, 'TRUE': 1, False: 0, True: 1})

# Convert '0' and '1' strings to numbers for "multi" and "biz", and leave numbers as they are
data['multi'] = data['multi'].replace({'0': 0, '1': 1}).astype(int)
data['biz'] = data['biz'].replace({'0': 0, '1': 1}).astype(int)

/var/folders/t2/yz2y5gb1397dsm77lnp4kz0m0000gn/T/ipykernel_81493/2180715776.py:9: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data['room_shared'] = data['room_shared'].replace({'FALSE': 0, 'TRUE': 1, False: 0, True: 1})
/var/folders/t2/yz2y5gb1397dsm77lnp4kz0m0000gn/T/ipykernel_81493/2180715776.py:10: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data['room_private'] = data['room_private'].replace({'FALSE': 0, 'TRUE': 1, False: 0, True: 1})
/var/folders/t2/yz2y5gb1397dsm77lnp4kz0m0000gn/T/ipykernel_81493/2180715776.py:14: Future

In [10]:
num_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

cat_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

bin_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent'))
])

In [11]:
preprocessor = ColumnTransformer([
    ('num', num_pipeline, numerical_features),
    ('cat', cat_pipeline, categorical_features),
    ('bin', bin_pipeline, binary_features),
])

In [12]:
X = data[numerical_features + categorical_features + binary_features]
y = data['log_price']
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [ ]:
xgb_model = xgb.XGBRegressor(
    n_estimators=1000,
    learning_rate=0.05,
    max_depth=8,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
    tree_method="gpu_hist",
    early_stopping_rounds=50
)